<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/from_gee_to_colab_archeology_28may2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Install geemap if not already installed
!pip install geemap



In [ ]:
# based on similar date
import geemap

# Define the region of interest (Eastern Brazil)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
brazil_geometry = mask.geometry()

# Clip to Eastern Brazil using longitude filtering
#east_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-40, -5, -30, -30]))  # Adjust as needed
southeast_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-55, -25, -40, -35]))  # Adjust as needed
# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-05-01'
end_date = '2024-05-31'
def classify_image(date):
    date = ee.Date(date)

    sentinel2 = mosaiced_sentinel2.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()
    sentinel1 = mosaiced_sentinel1.filter(ee.Filter.eq("date", date.format("YYYY-MM-dd"))).first()

    combined = sentinel2.addBands(sentinel1)

    classified = ee.Algorithms.If(
        archaeology_points.size().gt(0),
        combined.classify(
            ee.Classifier.smileRandomForest(10).train(
                combined.sampleRegions(collection=archaeology_points, properties=['class'], scale=30),
                'class'
            )
        ),
        ee.Image.constant(-9999).rename("classification")  # Placeholder if no training points
    )

    return ee.Image(classified).set("date", date.format("YYYY-MM-dd"))
### Step 1: Mosaic Sentinel-1 images per date ###
def mosaic_sentinel1(date):
    date = ee.Date(date)
    sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(southeast_brazil) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select("VV")

    return sentinel1_images.median().set("date", date.format("YYYY-MM-dd"))

### Step 2: Mosaic Sentinel-2 images per date ###
def mosaic_sentinel2(date):
    date = ee.Date(date)
    sentinel2_images = ee.ImageCollection("COPERNICUS/S2").filterBounds(southeast_brazil) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(["B8", "B11", "B12"])

    return sentinel2_images.median().set("date", date.format("YYYY-MM-dd"))

# Extract available dates
sentinel1_dates = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .aggregate_array('system:time_start').map(ee.Date)

sentinel2_dates = ee.ImageCollection("COPERNICUS/S2").filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .aggregate_array('system:time_start').map(ee.Date)

# Find matching dates between Sentinel-1 and Sentinel-2
formatted_sentinel1_dates = sentinel1_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
formatted_sentinel2_dates = sentinel2_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))

common_dates = formatted_sentinel1_dates.filter(ee.Filter.inList('item', formatted_sentinel2_dates))

# Create mosaiced collections
mosaiced_sentinel1 = ee.ImageCollection(common_dates.map(mosaic_sentinel1))
mosaiced_sentinel2 = ee.ImageCollection(common_dates.map(mosaic_sentinel2))

# Create Map
m = geemap.Map(center=[-20, -40], zoom=5)  # Centered around Eastern Brazil

# Display Sentinel-1 mosaiced images
list_s1 = mosaiced_sentinel1.toList(mosaiced_sentinel1.size())
for i in range(list_s1.size().getInfo()):
    img = ee.Image(list_s1.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
    m.addLayer(img, vis_params_s1, f"Sentinel-1 Mosaiced ({date_label})")

# Display Sentinel-2 mosaiced images
list_s2 = mosaiced_sentinel2.toList(mosaiced_sentinel2.size())
for i in range(list_s2.size().getInfo()):
    img = ee.Image(list_s2.get(i))
    date_label = img.get("date").getInfo()
    vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
    m.addLayer(img, vis_params_s2, f"Sentinel-2 Mosaiced ({date_label})")
'''classified_images = ee.ImageCollection(common_dates.map(classify_image))
print("Number of classified images:", classified_images.size().getInfo())
# Display the map
# Display Classified Images
list_classified = classified_images.toList(classified_images.size())
for i in range(list_classified.size().getInfo()):
    img = ee.Image(list_classified.get(i))
    date_label = img.get("date").getInfo()
    vis_params_classified = {
        "bands": ["classification"],
        "min": 0,
        "max": 1,
        "palette": ["red", "yellow", "green"]
    }
    m.addLayer(img, vis_params_classified, f"Classified Archaeological Predictions ({date_label})")'''
# Add the archaeology points layer
archaeology_vis = {
    'color': 'blue',
    'pointRadius': 5
}
m.addLayer(archaeology_points, archaeology_vis, 'Archaeology Points')

# Display the map
m


In [ ]:
import geemap

# Define the region of interest (Southeastern Brazil)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
brazil_geometry = mask.geometry()

# Clip to Southeastern Brazil using longitude filtering
southeast_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-55, -25, -40, -35]))  # Adjust as needed

# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-05-01'
end_date = '2024-05-31'

### Step 1: Mosaic Sentinel-1 images ###
sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select("VV")

mosaiced_sentinel1 = sentinel1_images.median()

### Step 2: Mosaic Sentinel-2 images ###
sentinel2_images = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .select(["B8", "B11", "B12"])

mosaiced_sentinel2 = sentinel2_images.median()

### Step 3: Combine Sentinel-1 & Sentinel-2 mosaics and classify ###
combined = mosaiced_sentinel2.addBands(mosaiced_sentinel1)

classifier = ee.Classifier.smileRandomForest(10)

# Ensure training data exists
training_data = combined.sampleRegions(
    collection=archaeology_points,
    properties=['class'],
    scale=10
)
training_size = training_data.size().getInfo()
if training_size == 0:
    print("Warning: Training data is empty. Check feature properties or location.")
else:
    print(f"Training data contains {training_size} samples.")

# Train classifier only if there are valid training points
classified = ee.Algorithms.If(
    training_data.size().gt(0),
    combined.classify(classifier.train(training_data, 'class')),
    ee.Image.constant(-9999).rename("classification")  # Placeholder if no training points exist
)


# Create Map
m = geemap.Map(center=[-25, -50], zoom=5)  # Centered around Southeastern Brazil

# Display Sentinel-1 mosaic
vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
m.addLayer(mosaiced_sentinel1, vis_params_s1, "Sentinel-1 Mosaic")

# Display Sentinel-2 mosaic
vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
m.addLayer(mosaiced_sentinel2, vis_params_s2, "Sentinel-2 Mosaic")

# Add the classified layer
vis_params_classification = {"min": 0, "max": 3, "palette": ["blue", "green", "yellow", "red"]}
m.addLayer(classified, vis_params_classification, "Classified Image")

# Add archaeology points layer
archaeology_vis = {
    'color': 'blue',
    'pointRadius': 5
}
m.addLayer(archaeology_points, archaeology_vis, "Archaeology Points")

# Display the map
m


In [ ]:
# based on location
import geemap
# Define region of interest
m = geemap.Map(center=[-14.2, -51.9], zoom=4)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
SA = mask.geometry()
m.addLayer(mask, {"color": "black"}, "South America Mask")
m

# Load archaeological points
archaeology_points = ee.FeatureCollection('users/mohamadawadlebanon/Archeologicalsites')

# Define date range
start_date = '2024-03-01'
end_date = '2024-04-30'

# Extract available dates for Sentinel-1
sentinel1_dates = ee.List(ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .aggregate_array('system:time_start')).map(ee.Date)

# Extract available dates for Sentinel-2 (with cloud cover < 10%)
sentinel2_dates = ee.List(ee.ImageCollection("COPERNICUS/S2").filterBounds(SA)
    .filterDate(start_date, end_date)
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .aggregate_array('system:time_start')).map(ee.Date)

# Find exact matching dates
#common_dates = sentinel1_dates.filter(lambda date: sentinel2_dates.contains(date))
#common_dates = sentinel1_dates.map(lambda date: ee.Algorithms.If(sentinel2_dates.contains(date), date, None)) \
#    .removeAll([None])
common_dates = sentinel1_dates.filter(ee.Filter.inList('item', sentinel2_dates))
#print(common_dates.size().getInfo())  # Should be > 0
formatted_dates = sentinel1_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
print("Sentinel-1 Dates:", formatted_dates.getInfo())
formatted_dates = sentinel2_dates.map(lambda date: ee.Date(date).format("YYYY-MM-dd"))
print("Sentinel-2 Dates:", formatted_dates.getInfo())



# Function to combine and classify images by matching date
def classify_image(date):
    date = ee.Date(date)

    # Load matching Sentinel-1 and Sentinel-2 images
    sentinel2 = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA) \
        .filterDate(date, date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(["B8", "B11", "B12"]).median()

    sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA) \
        .filterDate(date, date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select("VV").median()

    combined = sentinel2.addBands(sentinel1)

    # Train classifier using archaeological points
    training = combined.sampleRegions(collection=archaeology_points, properties=['class'], scale=30)
    classifier = ee.Classifier.smileRandomForest(10).train(training, 'class')

    # Classify image
    classified = combined.classify(classifier)

    return classified.set("date", date.format("YYYY-MM-dd"))  # Store date

# Apply classification to each image in the collection
classified_images = ee.ImageCollection(common_dates.map(classify_image))

# Create Map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Display each classified image separately
list_of_images = classified_images.toList(classified_images.size())

for i in range(list_of_images.size().getInfo()):
    img = ee.Image(list_of_images.get(i))
    date_label = img.get("date").getInfo()

    vis_params = {
        "bands": ["B12", "B11", "VV"],
        "min": 0,
        "max": 1,
        "gamma": 1.4
    }

    m.addLayer(img, vis_params, f"Archaeological Predictions ({date_label})")

m.addLayer(mask, {"color": "black"}, "South America Mask")
m


In [ ]:
!pip install /content/ee-packages-py-main/